# POS tagging and NER

Upload file

In [ ]:
from google.colab import files
uploaded = files.upload()

import io
import pandas as pd

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

  # Assuming the uploaded file is a CSV
  try:
    df_lion = pd.read_csv(io.BytesIO(uploaded[fn]))
    print("Successfully loaded CSV into df_lion")
  except pd.errors.ParserError:
      print(f"Error: Could not parse {fn} as a CSV. Please upload a valid CSV file.")
      df_lion = pd.DataFrame() # Create an empty DataFrame if parsing fails
  except Exception as e:
    print(f"An unexpected error occurred: {e}")
    df_lion = pd.DataFrame()

In [ ]:
print(df_lion.columns)

In [ ]:
df_lion

Install library and dependency

In [ ]:
!pip install nlp-id transformers torch tqdm
from nlp_id.postag import PosTag
from transformers import pipeline
from tqdm import tqdm
tqdm.pandas()

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


In [ ]:
# Initialize models
pos_tagger = PosTag()
ner_model = pipeline("ner", model="cahya/bert-base-indonesian-NER", grouped_entities=True)

Pos Tagging with nlp-id (normalized stemmed)

In [ ]:
# Make sure the text column is string
df_lion["text_final"] = df_lion["text_final"].astype(str)

# Apply POS tagging
df_lion["pos_tags"] = df_lion["text_final"].progress_apply(lambda x: pos_tagger.get_pos_tag(x))


In [ ]:
df_lion[["text_final", "pos_tags"]].head(10)

NER with cahya bert (normalized_text)

In [ ]:
# Function to extract named entities
def extract_hf_entities(text):
    try:
        results = ner_model(text)
        return [(r['word'], r['entity_group']) for r in results]
    except Exception as e:
        return []

df_lion["ner_tags"] = df_lion["text_final"].progress_apply(extract_hf_entities)


In [ ]:
df_lion[["text_final", "ner_tags"]].head(30)

In [ ]:
df_lion

In [ ]:
# Save the DataFrame to a CSV file
df_lion.to_csv('tagged_articles.csv', index=False)